Сверточная нейронная сеть (CNN)  
=================================
Для того, чтобы нейронная сеть решала сложные задачи, нельзя обойтись несколькими слоями, необходимо добавление многих новых слоев. А это влечет проблемы. Во-первых, сталкиваемся с проблемой забывания градиента, хотя это и можно решить при помощи чувствительной функции активации — семейства ReLU функций. Другая проблема с глубокими полностью соединенными сетями — количество весов для тренировки быстро растет. Это означает, что процесс тренировки замедляется или становится практически невыполнимым, а модель может переобучаться. 

Сверточные нейронные сети пытаются решить вторую проблему, используя корреляции между смежными входами в картинках или временных рядах.   
Реализуем нейросеть со следующим описанием:  
В самом начале на вход подаются черно-белые представления символов размером 28х28 пикселей каждое. Первый слой состоит из 32 каналов сверточных фильтров размера 5х5 + активационная функция ReLU, затем идет 2х2 max pooling с даунсемплингом с шагом 2 (этот слой выводит данные размером 14х14). На следующий слой подается выход с первого слоя размера 14х14, который сканируется снова 5х5 сверточными фильтрами с 64 каналов, затем следует 2х2 max pooling с даунсемплингом для генерирования выхода размером 7х7.

In [3]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
import torchvision.datasets
from bokeh.plotting import figure
from bokeh.io import show
from bokeh.models import LinearAxis, Range1d
import numpy as np

In [4]:
# Hyperparameters
num_epochs = 6
num_classes = 10
batch_size = 100
learning_rate = 0.001

DATA_PATH = 'C:\\Users\Andy\PycharmProjects\MNISTData'
MODEL_STORE_PATH = 'C:\\Users\Andy\PycharmProjects\pytorch_models\\'

In [5]:
# transforms to apply to the data
trans = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])

# MNIST dataset
train_dataset = torchvision.datasets.MNIST(root=DATA_PATH, train=True, transform=trans, download=True)
test_dataset = torchvision.datasets.MNIST(root=DATA_PATH, train=False, transform=trans)

# Data loader
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

Extracting C:\Users\Andy\PycharmProjects\MNISTData/MNIST/raw/train-images-idx3-ubyte.gz to C:\Users\Andy\PycharmProjects\MNISTData/MNIST/raw


Extracting C:\Users\Andy\PycharmProjects\MNISTData/MNIST/raw/train-labels-idx1-ubyte.gz to C:\Users\Andy\PycharmProjects\MNISTData/MNIST/raw


Extracting C:\Users\Andy\PycharmProjects\MNISTData/MNIST/raw/t10k-images-idx3-ubyte.gz to C:\Users\Andy\PycharmProjects\MNISTData/MNIST/raw




Extracting C:\Users\Andy\PycharmProjects\MNISTData/MNIST/raw/t10k-labels-idx1-ubyte.gz to C:\Users\Andy\PycharmProjects\MNISTData/MNIST/raw
Processing...
Done!


На этом шаге необходимо задать класс nn.Module, определяющий сверточную нейронную сеть, которую мы хотим обучить. Перед тренировкой модели создаем экземпляр (instance) нашего класса ConvNet(), определяем функцию потерь и оптимизатор. После тренировочного цикла выводим прогресс после каждых 100 итераций внутреннего цикла. 

In [7]:
# Convolutional neural network (two convolutional layers)
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.drop_out = nn.Dropout()
        self.fc1 = nn.Linear(7 * 7 * 64, 1000)
        self.fc2 = nn.Linear(1000, 10)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.drop_out(out)
        out = self.fc1(out)
        out = self.fc2(out)
        return out


model = ConvNet()

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Train the model
total_step = len(train_loader)
loss_list = []
acc_list = []
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # Run the forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss_list.append(loss.item())

        # Backprop and perform Adam optimisation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Track the accuracy
        total = labels.size(0)
        _, predicted = torch.max(outputs.data, 1)
        correct = (predicted == labels).sum().item()
        acc_list.append(correct / total)

        if (i + 1) % 100 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Accuracy: {:.2f}%'
                  .format(epoch + 1, num_epochs, i + 1, total_step, loss.item(),
                          (correct / total) * 100))

Epoch [1/6], Step [100/600], Loss: 0.1038, Accuracy: 97.00%
Epoch [1/6], Step [200/600], Loss: 0.1393, Accuracy: 95.00%
Epoch [1/6], Step [300/600], Loss: 0.1380, Accuracy: 95.00%
Epoch [1/6], Step [400/600], Loss: 0.1272, Accuracy: 94.00%
Epoch [1/6], Step [500/600], Loss: 0.1196, Accuracy: 95.00%
Epoch [1/6], Step [600/600], Loss: 0.0213, Accuracy: 100.00%
Epoch [2/6], Step [100/600], Loss: 0.0838, Accuracy: 97.00%
Epoch [2/6], Step [200/600], Loss: 0.0939, Accuracy: 98.00%
Epoch [2/6], Step [300/600], Loss: 0.1343, Accuracy: 96.00%
Epoch [2/6], Step [400/600], Loss: 0.1211, Accuracy: 97.00%
Epoch [2/6], Step [500/600], Loss: 0.0321, Accuracy: 98.00%
Epoch [2/6], Step [600/600], Loss: 0.2070, Accuracy: 96.00%
Epoch [3/6], Step [100/600], Loss: 0.0301, Accuracy: 98.00%
Epoch [3/6], Step [200/600], Loss: 0.0845, Accuracy: 96.00%
Epoch [3/6], Step [300/600], Loss: 0.0720, Accuracy: 97.00%
Epoch [3/6], Step [400/600], Loss: 0.0532, Accuracy: 98.00%
Epoch [3/6], Step [500/600], Loss: 0.02

Ниже происходит тестирование модели. Выводим результат. Точность составляет 99, 11%. Этот результат определенно лучше точности базовой полносвязной нейронной сети.

In [11]:
# Test the model
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Точность на тестовой выборке на 10000 картинках составляет: {} %'.format((correct / total) * 100))

torch.save(model.state_dict(), MODEL_STORE_PATH + 'conv_net_model.ckpt')

p = figure(y_axis_label='Loss', width=850, y_range=(0, 1), title='PyTorch ConvNet results')
p.extra_y_ranges = {'Accuracy': Range1d(start=0, end=100)}
p.add_layout(LinearAxis(y_range_name='Accuracy', axis_label='Accuracy (%)'), 'right')
p.line(np.arange(len(loss_list)), loss_list)
p.line(np.arange(len(loss_list)), np.array(acc_list) * 100, y_range_name='Accuracy', color='red')
show(p)

Точность на тестовой выборке на 10000 картинках составляет: 99.11 %
